In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
pc1 = pd.read_csv(r'/content/pc1_csv.csv')
print('pc1 shape: ', pc1.shape)
pc1.head()

pc1 shape:  (1109, 22)


loc   v(g)  ev(g)  iv(G)       N         V     L      D       I  \
0    1.1    1.4    1.4    1.4     1.3      1.30  1.30   1.30    1.30   
1    1.0    1.0    1.0    1.0     1.0      1.00  1.00   1.00    1.00   
2   91.0    9.0    3.0    2.0   318.0   2089.21  0.04  27.68   75.47   
3  109.0   21.0    5.0   18.0   381.0   2547.56  0.04  28.37   89.79   
4  505.0  106.0   41.0   82.0  2339.0  20696.93  0.01  75.93  272.58   

            E  ...  lOCode  lOComment  locCodeAndComment  lOBlank  uniq_Op  \
0        1.30  ...       2          2                  2        2      1.2   
1        1.00  ...       1          1                  1        1      1.0   
2    57833.24  ...      80         44                 11       31     29.0   
3    72282.68  ...      97         41                 12       24     28.0   
4  1571506.88  ...     457         71                 48       49     64.0   

   uniq_Opnd  total_Op  total_Opnd  branchCount  defects  
0        1.2       1.2         1.2          1.4    False  
1        1.0       1.0         1.0          1.0     True  
2       66.0     192.0       126.0         17.0     True  
3       75.0     229.0       152.0         38.0     True  
4      397.0    1397.0       942.0        178.0     True  

[5 rows x 22 columns]

In [5]:
pc1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1109 entries, 0 to 1108
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   loc                1109 non-null   float64
 1   v(g)               1109 non-null   float64
 2   ev(g)              1109 non-null   float64
 3   iv(G)              1109 non-null   float64
 4   N                  1109 non-null   float64
 5   V                  1109 non-null   float64
 6   L                  1109 non-null   float64
 7   D                  1109 non-null   float64
 8   I                  1109 non-null   float64
 9   E                  1109 non-null   float64
 10  B                  1109 non-null   float64
 11  T                  1109 non-null   float64
 12  lOCode             1109 non-null   int64  
 13  lOComment          1109 non-null   int64  
 14  locCodeAndComment  1109 non-null   int64  
 15  lOBlank            1109 non-null   int64  
 16  uniq_Op            1109 

In [6]:
target = 'defects'
y = pc1[target]
x=pc1.drop(['defects'], axis=1)
x.shape

(1109, 21)

after running " *from mlxtend.feature_selection import SequentialFeatureSelector as SFS* " this command you might get error this is because mlxtend is trying to import joblib from sklearn but it no longer exist there so fix this issue go to the file location (there will a file location link in the error message like this " /usr/local/lib/python3.7/dist-packages/mlxtend/feature_selection/sequential_feature_selector.py <module> " click on this location link)
then it will show you code written in that file then you just have to look for this line " from sklearn.externals.joblib import Parallel, delayed " and edit it to " from joblib import Parallel, delayed " and then save it by pressing ctrl+s. then run the code again. Again it will give error but this time the location link will be different and you have to again edit the file in same way as done priviouslly and save it and run the code again and this will solve the error.

In [7]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.svm import SVC
svc = SVC()


In [8]:
ffs = SFS(svc, # can use any any learning method 
          k_features='best', 
          forward=True, 
          floating=False, 
          verbose=2,
          scoring='accuracy',
          cv=5).fit(x, y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    1.4s finished

[2022-09-02 22:13:15] Features: 1/21 -- score: 0.9305694835106599[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.7s finished

[2022-09-02 22:13:16] Features: 2/21 -- score: 0.9305694835106599[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:    1.4s finished

[2022-09-02 22:13:18] Features: 3/21 -- score: 0.9305694835106599[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

In [9]:
ffs.k_feature_names_ 

('loc',)

In [10]:
x_reduced = ffs.transform(x)
x_reduced.shape

(1109, 1)

In [11]:
y.shape

(1109,)

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_reduced, y, test_size=0.3, random_state=10)
print(x.shape)
print(y.shape)
print('-------------------')
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1109, 21)
(1109,)
-------------------
(776, 1)
(776,)
(333, 1)
(333,)


In [13]:
from sklearn.svm import SVC
from sklearn import metrics #for checking the model accuracy
svc = SVC()
svc.fit(x_train, y_train)
svc_pred=svc.predict(x_test)
from sklearn import metrics #for checking the model accuracy
acc_svm = metrics.accuracy_score(svc_pred,y_test)
print('The accuracy of the SVM is:',acc_svm)
print('Th confusion matrix of SVM:\n ',metrics.confusion_matrix(y_test, svc_pred))

The accuracy of the SVM is: 0.9309309309309309
Th confusion matrix of SVM:
  [[310   0]
 [ 23   0]]


In [14]:
print(metrics.confusion_matrix(y_test, svc_pred))
print(metrics.classification_report(y_test, svc_pred))

[[310   0]
 [ 23   0]]
              precision    recall  f1-score   support

       False       0.93      1.00      0.96       310
        True       0.00      0.00      0.00        23

    accuracy                           0.93       333
   macro avg       0.47      0.50      0.48       333
weighted avg       0.87      0.93      0.90       333



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2) #put 2 neighbors because we have 2 labels.
knn.fit(x_train,y_train)
knn_pred = knn.predict(x_test)
acc_knn = metrics.accuracy_score(knn_pred,y_test)
print('The accuracy of the KNN is', acc_knn)
print('The confusion matrix of KNN:\n ',metrics.confusion_matrix(y_test, knn_pred))
print(metrics.confusion_matrix(y_test, knn_pred))
print(metrics.classification_report(y_test, knn_pred))

The accuracy of the KNN is 0.9309309309309309
The confusion matrix of KNN:
  [[310   0]
 [ 23   0]]
[[310   0]
 [ 23   0]]
              precision    recall  f1-score   support

       False       0.93      1.00      0.96       310
        True       0.00      0.00      0.00        23

    accuracy                           0.93       333
   macro avg       0.47      0.50      0.48       333
weighted avg       0.87      0.93      0.90       333



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
model_score = model.score(x_test, y_test)
print(model_score)
print(metrics.confusion_matrix(y_test, y_predict))
print(metrics.classification_report(y_test, y_predict))

0.933933933933934
[[310   0]
 [ 22   1]]
              precision    recall  f1-score   support

       False       0.93      1.00      0.97       310
        True       1.00      0.04      0.08        23

    accuracy                           0.93       333
   macro avg       0.97      0.52      0.52       333
weighted avg       0.94      0.93      0.90       333



In [17]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train, y_train)
model.score(x_train, y_train) 

0.9265463917525774

In [18]:
test_pred = model.predict(x_test)
print(metrics.classification_report(y_test, test_pred))
print(metrics.confusion_matrix(y_test, test_pred))

              precision    recall  f1-score   support

       False       0.94      0.98      0.96       310
        True       0.33      0.13      0.19        23

    accuracy                           0.92       333
   macro avg       0.64      0.56      0.57       333
weighted avg       0.90      0.92      0.91       333

[[304   6]
 [ 20   3]]
